# Deep Learning Project \#1

## Longer Runs

### Initial Part

In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt

Downloading the dataset:

In [2]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=True,download=True)
testdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=False,download=True)

Files already downloaded and verified
Files already downloaded and verified


Mean and Std calculation for normalization:

In [3]:
tr_mean = trainingdata.data.mean(axis = (0,1,2)) / 255
tr_std = trainingdata.data.std(axis = (0,1,2))  / 255
print(tr_mean)
print(tr_std)

[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


Data Augmentation (Random Transforms and Normalization)

In [4]:
tr_train = torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(size=[32,32], padding=4),
        torchvision.transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        torchvision.transforms.RandomRotation(10),
        torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(tr_mean, tr_std)
    ])

tr_test = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize(tr_mean, tr_std)])

Download the datasets

In [5]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=True,download=False,transform=tr_train)
testdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=False,download=False,transform=tr_test)

Check how many Training and Test Data we have

In [6]:
print(len(trainingdata))
print(len(testdata))

50000
10000


Printing size of the images:

In [7]:
image, label = trainingdata[13]
print(image.size(),label)

torch.Size([3, 32, 32]) 2


## Generic ResNet Model:

### We defined our model in a parametric way

So here we can set up our parameters later

N: Number of residual layers, 3-4-5  
C_1: # channels in the first layer  
num_blocks: Nx1 vector, num_blocks[i] is number of residual blocks in layer i  
conv_kernel_size: Nx1 vector, kernel size for conv layers in each residual layer  
skip_kernel_size: Nx1 vector, kernel size for skip connections between each residual block  
avg_pool_size: Kernel size for the average pooling at last layer 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, kernel_size, skip_kernel_size, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size, stride=stride, padding=int((kernel_size-1)/2), bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size,
                               stride=1, padding=int((kernel_size-1)/2), bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          skip_kernel_size, stride=stride, padding=int((skip_kernel_size-1)/2), bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = C_1
        self.avg_pool_size = avg_pool_size

        self.conv1 = nn.Conv2d(3, C_1, conv_kernel_size[0],
                               stride=1, padding=int((conv_kernel_size[0]-1)/2), bias=False)
        self.bn1 = nn.BatchNorm2d(C_1)

        res_layers = []
        
        for i in range(N):
          if i == 0:
            s = 1
          else:
            s = 2
          res_layers.append(self._make_layer(block, (2**i)*C_1, num_blocks[i], conv_kernel_size[i], skip_kernel_size[i], stride=s))

        self.layer = nn.Sequential(*res_layers)

        ps = 2**(np.ceil(np.log2(avg_pool_size)))


        self.linear = nn.Linear(int(C_1*32*32/(2**(N-1))/(ps**2)), num_classes)

    def _make_layer(self, block, planes, num_blocks, conv_kernel_size, skip_kernel_size, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, conv_kernel_size, skip_kernel_size, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer(out)
        out = F.avg_pool2d(out, self.avg_pool_size)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model(N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size):
    return ResNet(BasicBlock, N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size, num_classes=10)
    

### Parameters From Previous Runs in a List

In [9]:
Grid_Search = ((2,1,256), #1
              (2,2,172), #2
              (2,3,140), #3
              (2,4,120), #4
              (2,5,106), #5
              (2,6,96),  #6
              (2,7,90),  #7
              (3,1,128), #8
              (3,2,85),  #9
              (3,3,68),  #10
              (3,4,58),  #11
              (3,5,52),  #12
              (4,1,64),  #13
              (4,2,42),  #14
              (4,3,32),  #15
              (4,4,29),  #16
              (4,5,26),  #17
              (5,1,32),  #18
              (5,2,21),  #19
              (5,3,17),  #20
              (5,4,14),  #21
              (5,5,13))  #22

Our top picks are: 

1.   N=3 B=2 C=85 #9
2.   N=3 B=3 C=68 #10
3.   N=2 B=3 C=140 #3





### Function to call the model is defined here

In [10]:
import time

def neural_net(N,C1,num_blocks,conv_kernel_size,avg_pool_size,skip_kernel_size):
    net = project1_model(N,
                    num_blocks, 
                    C1, 
                    conv_kernel_size,
                    skip_kernel_size,
                    avg_pool_size).cuda()
    return net

## TRAINING (EDITS START FROM HERE)

Setting up the network

In [11]:
Ind = 8
N,B,C = Grid_Search[Ind]

num_blocks = [B]*N
skip_kernel_size = [1]*N
conv_kernel_size = [3]*N
avg_pool_size = int(32/(2**(N-1)))
net = neural_net(N,C,num_blocks,conv_kernel_size,avg_pool_size,skip_kernel_size)

Setting up optimizer and scheduler

In [12]:
lr = 0.01
step_size = 10
gamma = 0.90
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

We set up our data loaders.  
We can try different Batch Sizes here.

In [13]:
N_batch = 64

trainDataLoader = torch.utils.data.DataLoader(trainingdata,batch_size=N_batch,shuffle=True)
testDataLoader = torch.utils.data.DataLoader(testdata,batch_size=N_batch,shuffle=False)

### Summary

In [14]:
# Summary of the model
from torchinfo import summary
from torchvision import models
summary(net,(1,3,32,32))


# TRY THIS IF ABOVE GIVES ERROR
# from torchsummary import summary
# from torchvision import models
# summary(net,(3,32,32))


Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        --                        --
├─Conv2d: 1-1                                 [1, 85, 32, 32]           2,295
├─BatchNorm2d: 1-2                            [1, 85, 32, 32]           170
├─Sequential: 1-3                             [1, 340, 8, 8]            --
│    └─Sequential: 2-1                        [1, 85, 32, 32]           --
│    │    └─BasicBlock: 3-1                   [1, 85, 32, 32]           130,390
│    │    └─BasicBlock: 3-2                   [1, 85, 32, 32]           130,390
│    └─Sequential: 2-2                        [1, 170, 16, 16]          --
│    │    └─BasicBlock: 3-3                   [1, 170, 16, 16]          405,620
│    │    └─BasicBlock: 3-4                   [1, 170, 16, 16]          520,880
│    └─Sequential: 2-3                        [1, 340, 8, 8]            --
│    │    └─BasicBlock: 3-5                   [1, 340, 8, 8]           

### Training

In [16]:
import time


train_loss_history = []
test_loss_history = []
train_acc_history = []
test_acc_history = []

Nepoch = 300

for epoch in range(Nepoch): 
    train_loss = 0.0
    test_loss = 0.0
    t1 = time.time()
    train_acc = 0.0
    test_acc = 0.0

    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        predicted_output = net(images)
        fit = Loss(predicted_output,labels)
        fit.backward()
        optimizer.step()   
        train_loss += fit.item()
        train_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

    for i, data in enumerate(testDataLoader):
        with torch.no_grad():
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            test_loss += fit.item()
            test_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

    scheduler.step()
    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    train_acc = train_acc/len(trainingdata)
    test_acc = test_acc/len(testdata)
    train_acc_history.append(train_acc)
    test_acc_history.append(test_acc)
    t2 = time.time()
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)

    print(f'Epoch: {epoch} \tTrain Loss: {train_loss:.5f} \tTrain Acc: {train_acc:.4f} \tTest Loss: {test_loss:.5f} \tTest Acc: {test_acc:.4f} \tTime: {t2-t1:.4f}')

Epoch: 0 	Train Loss: 1.78522 	Train Acc: 0.3337 	Test Loss: 1.52942 	Test Acc: 0.4400 	Time: 72.2150
Epoch: 1 	Train Loss: 1.33652 	Train Acc: 0.5177 	Test Loss: 1.29147 	Test Acc: 0.5469 	Time: 68.0801
Epoch: 2 	Train Loss: 1.09655 	Train Acc: 0.6111 	Test Loss: 1.04110 	Test Acc: 0.6321 	Time: 68.2179
Epoch: 3 	Train Loss: 0.95265 	Train Acc: 0.6637 	Test Loss: 0.92246 	Test Acc: 0.6808 	Time: 68.1530
Epoch: 4 	Train Loss: 0.83932 	Train Acc: 0.7058 	Test Loss: 0.86679 	Test Acc: 0.7028 	Time: 68.3162
Epoch: 5 	Train Loss: 0.74687 	Train Acc: 0.7385 	Test Loss: 0.74028 	Test Acc: 0.7431 	Time: 68.1919
Epoch: 6 	Train Loss: 0.67665 	Train Acc: 0.7642 	Test Loss: 0.72721 	Test Acc: 0.7428 	Time: 68.1309
Epoch: 7 	Train Loss: 0.61589 	Train Acc: 0.7861 	Test Loss: 0.61843 	Test Acc: 0.7876 	Time: 67.7611
Epoch: 8 	Train Loss: 0.57880 	Train Acc: 0.7992 	Test Loss: 0.59258 	Test Acc: 0.7931 	Time: 68.1160
Epoch: 9 	Train Loss: 0.53660 	Train Acc: 0.8132 	Test Loss: 0.53408 	Test Acc: 0.

Epoch: 80 	Train Loss: 0.05596 	Train Acc: 0.9813 	Test Loss: 0.37625 	Test Acc: 0.9069 	Time: 68.3496
Epoch: 81 	Train Loss: 0.05764 	Train Acc: 0.9804 	Test Loss: 0.38576 	Test Acc: 0.9051 	Time: 68.1995
Epoch: 82 	Train Loss: 0.05330 	Train Acc: 0.9819 	Test Loss: 0.40400 	Test Acc: 0.9064 	Time: 68.2944
Epoch: 83 	Train Loss: 0.05291 	Train Acc: 0.9815 	Test Loss: 0.36946 	Test Acc: 0.9075 	Time: 68.3051
Epoch: 84 	Train Loss: 0.05324 	Train Acc: 0.9818 	Test Loss: 0.40175 	Test Acc: 0.9070 	Time: 68.3204
Epoch: 85 	Train Loss: 0.05325 	Train Acc: 0.9820 	Test Loss: 0.38824 	Test Acc: 0.9083 	Time: 68.2323
Epoch: 86 	Train Loss: 0.05082 	Train Acc: 0.9824 	Test Loss: 0.37504 	Test Acc: 0.9087 	Time: 68.2561
Epoch: 87 	Train Loss: 0.05081 	Train Acc: 0.9822 	Test Loss: 0.38738 	Test Acc: 0.9123 	Time: 68.2191
Epoch: 88 	Train Loss: 0.05043 	Train Acc: 0.9830 	Test Loss: 0.36981 	Test Acc: 0.9097 	Time: 68.1418
Epoch: 89 	Train Loss: 0.05008 	Train Acc: 0.9825 	Test Loss: 0.41778 	Te

Epoch: 159 	Train Loss: 0.01709 	Train Acc: 0.9941 	Test Loss: 0.41550 	Test Acc: 0.9163 	Time: 68.1827
Epoch: 160 	Train Loss: 0.01863 	Train Acc: 0.9938 	Test Loss: 0.40309 	Test Acc: 0.9201 	Time: 67.9982
Epoch: 161 	Train Loss: 0.01684 	Train Acc: 0.9939 	Test Loss: 0.41879 	Test Acc: 0.9163 	Time: 68.2328
Epoch: 162 	Train Loss: 0.01743 	Train Acc: 0.9944 	Test Loss: 0.42704 	Test Acc: 0.9201 	Time: 68.2370
Epoch: 163 	Train Loss: 0.01790 	Train Acc: 0.9939 	Test Loss: 0.41083 	Test Acc: 0.9186 	Time: 68.4048
Epoch: 164 	Train Loss: 0.01781 	Train Acc: 0.9943 	Test Loss: 0.41055 	Test Acc: 0.9183 	Time: 68.2622
Epoch: 165 	Train Loss: 0.01737 	Train Acc: 0.9944 	Test Loss: 0.41074 	Test Acc: 0.9179 	Time: 68.2654
Epoch: 166 	Train Loss: 0.01720 	Train Acc: 0.9944 	Test Loss: 0.42337 	Test Acc: 0.9170 	Time: 68.2485
Epoch: 167 	Train Loss: 0.01706 	Train Acc: 0.9944 	Test Loss: 0.41821 	Test Acc: 0.9161 	Time: 68.3817
Epoch: 168 	Train Loss: 0.01903 	Train Acc: 0.9939 	Test Loss: 0

Epoch: 238 	Train Loss: 0.00882 	Train Acc: 0.9971 	Test Loss: 0.44233 	Test Acc: 0.9200 	Time: 65.8979
Epoch: 239 	Train Loss: 0.00807 	Train Acc: 0.9972 	Test Loss: 0.45019 	Test Acc: 0.9184 	Time: 65.8929
Epoch: 240 	Train Loss: 0.00735 	Train Acc: 0.9976 	Test Loss: 0.44341 	Test Acc: 0.9195 	Time: 65.9224
Epoch: 241 	Train Loss: 0.00769 	Train Acc: 0.9973 	Test Loss: 0.44610 	Test Acc: 0.9178 	Time: 65.9007
Epoch: 242 	Train Loss: 0.00772 	Train Acc: 0.9976 	Test Loss: 0.44431 	Test Acc: 0.9189 	Time: 65.9864
Epoch: 243 	Train Loss: 0.00821 	Train Acc: 0.9971 	Test Loss: 0.44196 	Test Acc: 0.9195 	Time: 65.8807
Epoch: 244 	Train Loss: 0.00766 	Train Acc: 0.9974 	Test Loss: 0.44460 	Test Acc: 0.9203 	Time: 65.9586
Epoch: 245 	Train Loss: 0.00814 	Train Acc: 0.9975 	Test Loss: 0.44168 	Test Acc: 0.9213 	Time: 65.9537
Epoch: 246 	Train Loss: 0.00744 	Train Acc: 0.9973 	Test Loss: 0.44529 	Test Acc: 0.9195 	Time: 65.9921
Epoch: 247 	Train Loss: 0.00826 	Train Acc: 0.9973 	Test Loss: 0

In [20]:
for epoch in range(Nepoch,Nepoch+200): 
    train_loss = 0.0
    test_loss = 0.0
    t1 = time.time()
    train_acc = 0.0
    test_acc = 0.0

    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        predicted_output = net(images)
        fit = Loss(predicted_output,labels)
        fit.backward()
        optimizer.step()   
        train_loss += fit.item()
        train_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

    for i, data in enumerate(testDataLoader):
        with torch.no_grad():
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            test_loss += fit.item()
            test_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

    scheduler.step()
    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    train_acc = train_acc/len(trainingdata)
    test_acc = test_acc/len(testdata)
    train_acc_history.append(train_acc)
    test_acc_history.append(test_acc)
    t2 = time.time()
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)

    print(f'Epoch: {epoch} \tTrain Loss: {train_loss:.5f} \tTrain Acc: {train_acc:.4f} \tTest Loss: {test_loss:.5f} \tTest Acc: {test_acc:.4f} \tTime: {t2-t1:.4f}')

Epoch: 300 	Train Loss: 0.00608 	Train Acc: 0.9980 	Test Loss: 0.44175 	Test Acc: 0.9239 	Time: 64.9735
Epoch: 301 	Train Loss: 0.00497 	Train Acc: 0.9983 	Test Loss: 0.44050 	Test Acc: 0.9242 	Time: 61.9716
Epoch: 302 	Train Loss: 0.00645 	Train Acc: 0.9978 	Test Loss: 0.44504 	Test Acc: 0.9241 	Time: 62.0401
Epoch: 303 	Train Loss: 0.00626 	Train Acc: 0.9979 	Test Loss: 0.43716 	Test Acc: 0.9251 	Time: 62.1443
Epoch: 304 	Train Loss: 0.00540 	Train Acc: 0.9982 	Test Loss: 0.44094 	Test Acc: 0.9244 	Time: 65.4495
Epoch: 305 	Train Loss: 0.00620 	Train Acc: 0.9979 	Test Loss: 0.44247 	Test Acc: 0.9239 	Time: 66.7454
Epoch: 306 	Train Loss: 0.00582 	Train Acc: 0.9980 	Test Loss: 0.44460 	Test Acc: 0.9226 	Time: 66.6471
Epoch: 307 	Train Loss: 0.00556 	Train Acc: 0.9982 	Test Loss: 0.44018 	Test Acc: 0.9242 	Time: 66.6326
Epoch: 308 	Train Loss: 0.00553 	Train Acc: 0.9981 	Test Loss: 0.43835 	Test Acc: 0.9249 	Time: 66.6868
Epoch: 309 	Train Loss: 0.00483 	Train Acc: 0.9985 	Test Loss: 0

Epoch: 379 	Train Loss: 0.00409 	Train Acc: 0.9984 	Test Loss: 0.44529 	Test Acc: 0.9238 	Time: 68.7467
Epoch: 380 	Train Loss: 0.00470 	Train Acc: 0.9983 	Test Loss: 0.44033 	Test Acc: 0.9236 	Time: 68.6571
Epoch: 381 	Train Loss: 0.00383 	Train Acc: 0.9988 	Test Loss: 0.44122 	Test Acc: 0.9237 	Time: 68.7316
Epoch: 382 	Train Loss: 0.00370 	Train Acc: 0.9988 	Test Loss: 0.44548 	Test Acc: 0.9243 	Time: 68.5897
Epoch: 383 	Train Loss: 0.00382 	Train Acc: 0.9987 	Test Loss: 0.44310 	Test Acc: 0.9233 	Time: 68.7006
Epoch: 384 	Train Loss: 0.00359 	Train Acc: 0.9989 	Test Loss: 0.44318 	Test Acc: 0.9242 	Time: 68.7163
Epoch: 385 	Train Loss: 0.00387 	Train Acc: 0.9986 	Test Loss: 0.44663 	Test Acc: 0.9239 	Time: 68.7296
Epoch: 386 	Train Loss: 0.00375 	Train Acc: 0.9989 	Test Loss: 0.44936 	Test Acc: 0.9232 	Time: 68.6231
Epoch: 387 	Train Loss: 0.00388 	Train Acc: 0.9988 	Test Loss: 0.44834 	Test Acc: 0.9226 	Time: 68.5079
Epoch: 388 	Train Loss: 0.00382 	Train Acc: 0.9988 	Test Loss: 0

Epoch: 458 	Train Loss: 0.00317 	Train Acc: 0.9989 	Test Loss: 0.44508 	Test Acc: 0.9245 	Time: 67.5027
Epoch: 459 	Train Loss: 0.00335 	Train Acc: 0.9989 	Test Loss: 0.44644 	Test Acc: 0.9254 	Time: 67.4409
Epoch: 460 	Train Loss: 0.00337 	Train Acc: 0.9990 	Test Loss: 0.44523 	Test Acc: 0.9249 	Time: 67.4284
Epoch: 461 	Train Loss: 0.00374 	Train Acc: 0.9987 	Test Loss: 0.44504 	Test Acc: 0.9253 	Time: 67.3982
Epoch: 462 	Train Loss: 0.00378 	Train Acc: 0.9989 	Test Loss: 0.44498 	Test Acc: 0.9260 	Time: 67.4619
Epoch: 463 	Train Loss: 0.00445 	Train Acc: 0.9986 	Test Loss: 0.44533 	Test Acc: 0.9260 	Time: 67.4788
Epoch: 464 	Train Loss: 0.00315 	Train Acc: 0.9989 	Test Loss: 0.44669 	Test Acc: 0.9254 	Time: 67.4862
Epoch: 465 	Train Loss: 0.00330 	Train Acc: 0.9989 	Test Loss: 0.44467 	Test Acc: 0.9249 	Time: 67.3571
Epoch: 466 	Train Loss: 0.00296 	Train Acc: 0.9991 	Test Loss: 0.44516 	Test Acc: 0.9257 	Time: 67.3818
Epoch: 467 	Train Loss: 0.00354 	Train Acc: 0.9989 	Test Loss: 0

In [ ]:
np.savetxt('Train_Acc_Run_9_Batch_64_fast.txt',train_acc_history)
np.savetxt('Test_Acc_Run_9_Batch_64_fast.txt',test_acc_history)
np.savetxt('Train_Loss_Run_9_Batch_64_fast.txt',train_loss_history)
np.savetxt('Test_Loss_Run_9_Batch_64_fast.txt',test_loss_history)

In [ ]:
import pandas as pd

df = pd.DataFrame (train_acc_history)
filepath = 'Train_Acc_Run_9_Batch_64_fast.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (test_acc_history)
filepath = 'Test_Acc_Run_9_Batch_64_fast.xlsx'
df.to_excel(filepath, index=False)


df = pd.DataFrame (train_loss_history)
filepath = 'Train_Loss_Run_9_Batch_64_fast.xlsx'
df.to_excel(filepath, index=False)


df = pd.DataFrame (test_loss_history)
filepath = 'Test_Loss_Run_9_Batch_64_fast.xlsx'
df.to_excel(filepath, index=False)

In [ ]:
model_path = './project1_model_run_9_batch_64_fast.pt'
torch.save(net.state_dict(), model_path)